In [ ]:

from dask.distributed import Client
import dask.dataframe as dd
from dask_ml.model_selection import train_test_split

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import pandas as pd

In [ ]:
client = Client() 

In [ ]:
dt_file = './../data/test-parsed.csv'
df_file = './../data/train-parsed.csv'

dt = dd.read_csv(dt_file, sep="," ) # Archiu de test processat
df = dd.read_csv(df_file, sep="," ) # Arxiu de train processat

dt = dt.drop(columns=["ID"]) # Traiem la columna ID del test
df = df.dropna()
df.head()

In [ ]:
# AIXO NOMES CAL PER QUAN PROVEM AMB TEST.CSV
dt = dt[df.columns] # Ens assegurem que les columnes del test siguin les mateixes que les del train, i estiguin en el mateix ordre
dt.head()

In [ ]:
#Comproebem els tipus de dades (només int64 i float64)
df.dtypes

In [ ]:
train=False

if train:
    X = df.drop(columns=["weekly_demand"]) # Variables independents, totes excepte la que volem predir
    y = df[ ['weekly_demand']] # Variable que volem predir
else: # Si no entrenem, preparem les dades de test
    X = dt.drop(columns=["weekly_demand"]) # Variables independents, totes excepte la que volem predir i l'ID
    # la y no cal pq es la que volem predir


X.head()

In [ ]:
X_np = X.compute()
if train:
    y_np = y.compute()#.ravel()

In [ ]:
#AIXO NOMES CAL EXECUTAR-LO SI ESTEM ENTRENANT
if train:
    model = RandomForestRegressor(n_estimators=200, n_jobs=-1, random_state=42)
    model.fit(X_np, y_np)

In [ ]:
# Per guardar el model entrenat
import os
import joblib

os.makedirs("./../models", exist_ok=True)
model_path = "./../models/rf_model_v2.joblib"
joblib.dump(model, model_path, compress=3) 
print("Modelo guardado en:", model_path)

In [ ]:
# Per carregar el model entrenat
import joblib
model = joblib.load("./../models/rf_model_v2.joblib")
# usar model_loaded.predict(...)

In [ ]:
# Predicció
y_pred = model.predict(X_np)



In [ ]:
# Aqui creem una copia del CSV original, pero amb les dades predites


import numpy as np
import pandas as pd


y_pred_full = y_pred.ravel()
y_pred_int = np.rint(y_pred_full).astype(int)
y_pred_int = np.clip(y_pred_int, 0, None)

# llegir CSV original
df_orig = pd.read_csv(dt_file if not train else df_file)
assert len(df_orig) == len(y_pred_int), "Mismatch entre filas originales y predicciones"

# afegir columna amb prediccions
df_orig['weekly_demand'] = y_pred_int

# guardar nuevo CSV
out_path = './../data/test-parsed-with-predictions-rf.csv' if not train else './../data/train-parsed-with-predictions-rf.csv'
df_orig.to_csv(out_path, index=False)
print("Saved:", out_path)

In [ ]:

# Aixo serveix per avaluar les prediccions (si estem en mode train)
if train:
    df_p = df_orig.copy()
    df_p = df_p[["ID",'weekly_demand']]

    df_p["predicted_weekly_demand"] =  1*y_pred_int
    df_s = df_p.groupby('ID', as_index=False)['weekly_demand'].sum()
    df_s["predicted_weekly_demand"] = df_p.groupby('ID', as_index=False)['predicted_weekly_demand'].sum()["predicted_weekly_demand"]

    df_s["lost_sales"] = np.maximum(0, df_s["weekly_demand"] - df_s["predicted_weekly_demand"])
    df_s["excess stock"] = np.maximum(0, df_s["predicted_weekly_demand"] - df_s["weekly_demand"])

    average_lost_sales = df_s["lost_sales"].mean()
    average_excess_stock = df_s["excess stock"].mean()
    pd.set_option('display.max_rows', 20)
    df_s.head(200)
    #print("Average lost sales:", average_lost_sales)
    #print("Average excess stock:", average_excess_stock) 
